In [2]:
%matplotlib widget

In [3]:
import numpy as np
import pywt
from scipy.signal import butter, sosfiltfilt, find_peaks, detrend
from scipy.integrate import cumtrapz
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [4]:
def get_contact_points(vaccel):
    vacc = detrend(vaccel)
    
    sos = butter(4, 2 * 20 / 50, output='sos')
    fvacc = sosfiltfilt(sos, vacc)
    
    vpos = cumtrapz(fvacc, dx=1/50, initial=0)
    
    # diff 1
    coef1, freq1 = pywt.cwt(vpos, np.arange(1, 50), 'gaus1', sampling_period=1/50)
    
    # get most prominent frequency
    F = np.abs(np.fft.rfft(coef1[8]))
    f_step = np.fft.rfftfreq(coef1.shape[1])[np.argmax(F)] * 50
    
    f_ic_opt = 0.69 * f_step + 0.34
    f_fc_opt = 3.6 * f_step - 4.5
    
    idx1 = np.argmin(np.abs(freq1 - f_ic_opt))
    idx2 = np.argmin(np.abs(freq1 - f_fc_opt))
    
    vacc_id = coef1[idx1]
    ic, *_ = find_peaks(vacc_id, height=0.5 * np.std(vacc_id))
    
    coef2, _ = pywt.cwt(coef1[idx2], idx2, 'gaus1')
    vacc_idd = coef2[0]
    
    fc, *_ = find_peaks(vacc_idd, height=0.5*np.std(vacc_idd))
    
    return ic, fc, -vacc_id, vacc_idd, fvacc

In [64]:
def moe_nilsson(accel, v=1, ap=0, ml=2):
    sin_ap = np.mean(accel[:, ap])
    cos_ap = np.cos(np.arcsin(sin_ap))
    sin_ml = np.mean(accel[:, ml])
    cos_ml = np.cos(np.arcsin(sin_ml))
    
    sign_v = np.sign(np.mean(accel[:, v]))
    
    a_AP = (accel[:, ap] * cos_ap) - (np.abs(accel[:, v]) * sin_ap)  # use abs to let the sin_ap account for the necessary sign
    a_vp = (sign_v * accel[:, ap] * sin_ap) + (accel[:, v] * cos_ap)  # need the sign of vert accel to correctly account
    a_ML = (accel[:, ml] * cos_ml) - (np.abs(a_vp) * sin_ml)  # abs again to let sin_ml account for the necessary sign
    a_V = (sign_v * accel[:, ml] * sin_ml) + (a_vp * cos_ml) - sign_v  # sign of vert accel again to correctly add or subtract
    
    return a_V, a_AP, a_ML

In [20]:
data = np.loadtxt('sample_data/sample2.csv', delimiter=',')
ftime, facc = data[:, 0], data[:, 1:4]
ftime -= ftime[0]

In [21]:
f, ax = plt.subplots(figsize=(6, 3))
ax.plot(facc)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
i1, i2 = 21000, 28000
i1, i2 = 8350, 9850  # sample 2
ds = 4
ds = 2  # sample 2
time, acc = ftime[i1:i2:ds], facc[i1:i2:ds]  # downsample to 50Hz as well

In [23]:
ic, fc, va_id, va_idd, vaf = get_contact_points(acc[:, 1])

In [24]:
f, ax = plt.subplots(figsize=(8, 4))

ax.plot(time, vaf, label='Accel.')
ax.plot(time, va_id, label='ID')
ax.plot(time, va_idd, label='IDD')
ax.plot(time[ic], va_id[ic], 'ko', label='IC')
ax.plot(time[fc], va_idd[fc], 'k+', label='FC')

ax.legend()
f.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# GeneActiv/GaitMat Data

In [25]:
import inertial_sensor_routines as isr

In [26]:
path = 'share_data/TestACH_20200305_PKMAS_sync3pass.txt'

pkm = pd.read_csv(path, delimiter=';', skiprows=11)

ts = 'Time (sec.)'
pksum = np.zeros(pkm[ts].unique().size)

for i, t in enumerate(pkm[ts].unique()):
    mask = pkm[ts] == t
    
    pksum[i] = pkm.loc[mask, 'Level'].sum()
    
ff = pd.read_csv('share_data/TestACH_20200303_PKMAS_Footfalls.csv', skiprows=10, header=None, names=['Sample', 'Pass:side', 'First Contact', 'Last Contact'])

ff['Side'] = ff['Pass:side'].apply(lambda i: i.split(' ')[1])

In [62]:
df = isr.gnactv.tabular_conversion('share_data/TestACH20200305-PKMA_back_048877_2020-03-05 14-40-44.bin')

time = df.index.values.astype(int) / 1e9
acc = df[['X', 'Y', 'Z']].values
macc = np.linalg.norm(acc, axis=1)

gn_time = time-time[2420] + pkm[ts][0]

In [65]:
f, ax = plt.subplots(3, figsize=(10, 6), sharex=True)

v = 1
ap = 2
ml = 0

for i in range(3):
    ax[i].plot(gn_time[4000:5700], acc[4000:5700, i], color=f'C{i}')
    
walking = [(4075, 4450), (4550, 4850), (4950, 5300)]

for span in walking:
    a_v, a_ap, a_ml = moe_nilsson(acc[span[0]:span[1]], v, ap, ml)
    
    ax[v].plot(gn_time[span[0]:span[1]], a_v, color=f'C{v+1}')
    ax[ap].plot(gn_time[span[0]:span[1]], a_ap, color=f'C{ap+1}')
    ax[ml].plot(gn_time[span[0]:span[1]], a_ml, color=f'C{ml+1}')


f.tight_layout()
f.subplots_adjust(hspace=0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
acc_v, acc_ap, acc_ml = moe_nilsson(acc[4000:5500], v=1, ap=2, ml=0)

ic, fc, va_id, va_idd, vaf = get_contact_points(acc_v)

In [67]:
pkm_ic_l, pkm_ic_r, pkm_fc_l, pkm_fc_r = [], [], [], []

for c_ in ff[['First Contact', 'Last Contact', 'Side']].iterrows():
    if c_[1][2] == 'Left':
        pkm_ic_l.append(np.argmin(np.abs(gn_time - c_[1][0])))
        pkm_fc_l.append(np.argmin(np.abs(gn_time - c_[1][1])))
    elif c_[1][2] == 'Right':
        pkm_ic_r.append(np.argmin(np.abs(gn_time - c_[1][0])))
        pkm_fc_r.append(np.argmin(np.abs(gn_time - c_[1][1])))

pkm_ic_l = np.array(pkm_ic_l)
pkm_ic_r = np.array(pkm_ic_r)
pkm_fc_l = np.array(pkm_fc_l)
pkm_fc_r = np.array(pkm_fc_r)

In [68]:
sos = butter(4, [0.5/25, 10/25], output='sos', btype='band')

fp_acc = sosfiltfilt(sos, acc_ml)

fp_vel = cumtrapz(fp_acc, dx=1/50, initial=0)
fp_pos = cumtrapz(fp_vel, dx=1/50, initial=0)

fp_cf, _ = pywt.cwt(fp_pos, np.arange(1, 65), 'gaus1', sampling_period=1/50)
fp_cf2, _ = pywt.cwt(fp_cf[16], np.arange(1, 65), 'gaus1', sampling_period=1/50)

In [71]:
plt.close('all')

f, ax = plt.subplots(1, figsize=(10, 5))

ax.plot(gn_time[4000:5500], acc_v, color='C0')
# ax.plot(gn_time[4000:5500], fp_acc, color='C1', alpha=0.75)
# ax.plot(gn_time[4000:5500], fp_vel*5, color='C2', alpha=0.75)
ax.plot(gn_time[4000:5500], fp_pos*50, color='C3', alpha=0.75)

ax.set_ylabel('Vertical Acceleration [g]', color='C0')
ax.tick_params(axis='y', colors='C0')

# axx = ax.twinx()
# axx.grid(False)

# axx.plot(gn_time[4000:5500], va_id, color='C1', alpha=0.5)
# axx.plot(gn_time[4000:5500], va_idd, color='C2', alpha=0.5)


# plot the PKM event detections
ax.plot(gn_time[pkm_ic_l], acc_v[pkm_ic_l-4000], '+', color='green', markersize=6)
ax.plot(gn_time[pkm_fc_l], acc_v[pkm_fc_l-4000], 'x', color='green', markersize=6)

ax.plot(gn_time[pkm_ic_r], acc_v[pkm_ic_r-4000], '+', color='k', markersize=6)
ax.plot(gn_time[pkm_fc_r], acc_v[pkm_fc_r-4000], 'x', color='k', markersize=6)

# plot the acc event detections
ax.plot(gn_time[ic+4000], acc_v[ic], 'o', color='C1', markerfacecolor='none')
ax.plot(gn_time[fc+4000], acc_v[fc], 'o', color='C1', markerfacecolor='none')

ax.plot([gn_time[0], gn_time[-1]], [np.mean(acc_v)]*2, '--', alpha=0.35)


ax.set_xlim([36., 44.5])
ax.set_ylim([-0.85, 0.5])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.85, 0.5)

In [72]:
def t1(a=None, b=None, **kwargs):
    print(a, b)
    for key in kwargs:
        print(key, kwargs[key])